In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# Embedding size
m = 64

# Hidden size
hidden_size = 128

# Stack initialized?
init = False

In [3]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [4]:
s = tf.constant([], shape=(1,0), dtype=tf.float32, name='Strengths')
V = tf.constant([], shape=(m,0), dtype=tf.float32, name='Stack')

In [5]:
# Pop operation helpers
def check(stack, strengths, remaining, read, idx):
    # Bottom of stack
    return idx >= 0 #tf.logical_and(idx >= 1, remaining != 0)

def update(stack, strengths, remaining, read, idx):
    # Amount we can use at this step
    this_qty = tf.minimum(remaining, strengths[:,idx])

    # Update read value
    read = tf.reshape(read + this_qty * V[:,idx], tf.shape(read))  # for shape constraints

    # Update remaining strength
    remaining = tf.reshape(remaining - this_qty, tf.shape(remaining))

    # Update strengths
    before = strengths[:,:idx]
    this   = tf.reshape(tf.sub(strengths[:,idx], this_qty), (1, 1))
    after  = strengths[:,idx+1:]

    strengths = tf.reshape(tf.concat(1, [before, this, after]), tf.shape(strengths), name="strength_cat")

    # Update index
    idx = idx - 1

    return (stack, strengths, remaining, read, idx)

In [6]:
def symbolic_stack_update(d, u, v):
    '''
    Performs an update to the neural stack.
    
    Args:
      d: Push probability.
      u: Pop probability.
      v: Push value.
    
    Returns:
      r: The value read from the stack.
    '''
    global s, V, m, init
    
    if init:
        read0     = tf.zeros(64,)                # Read value
        idx0      = tf.shape(V)[1] - 1           # Index into the stack
        
        initialization = (V, s, u, read0, idx0)
        pop_operation = tf.while_loop(check, update, initialization)
        
        # Update strengths and perform read
        s = pop_operation[1]
        r = pop_operation[3]
        
    else:
        r = tf.zeros((m), dtype=np.float32)
        init = True
        pop_operation = None
    
    # Perform push
    V = tf.concat(1, [V, v])
    s = tf.concat(1, [s, d])
    
    return r, pop_operation

In [7]:
reads = []
pops  = []
num_ops = 5

d_ = [tf.placeholder(tf.float32, shape=(1, 1), name="d_%i" % t) for t in range(num_ops)]
u_ = [tf.placeholder(tf.float32, shape=(1, 1), name="u_%i" % t) for t in range(num_ops)]
v_ = [tf.placeholder(tf.float32, shape=(m, 1), name="v_%i" % t) for t in range(num_ops)]

for t in range(num_ops):
    read_t, pop_t = symbolic_stack_update(d_[t], u_[t], v_[t])
    reads.append(read_t)
    pops.append(pop_t)

In [8]:
feed_dict = {d_[t]: np.reshape(1.0, (1, 1)).astype(np.float) for t in range(num_ops)}
feed_dict.update({u_[0]: np.reshape(0.0, (1, 1)).astype(np.float)})
feed_dict.update({u_[t]: np.reshape(1.0, (1, 1)).astype(np.float) for t in range(1, num_ops)})
feed_dict.update({v_[t]: np.reshape(np.eye(m)[t], (64, 1)).astype(np.float) for t in range(num_ops)})

reads_t = sess.run(reads, feed_dict)

for read_t in reads_t:
    print read_t[0:10].astype(np.int32)
    print ''

[0 0 0 0 0 0 0 0 0 0]

[1 0 0 0 0 0 0 0 0 0]

[0 1 0 0 0 0 0 0 0 0]

[0 0 1 0 0 0 0 0 0 0]

[0 0 0 1 0 0 0 0 0 0]

